<a href="https://colab.research.google.com/github/ErnestoCash/ASAP/blob/main/EmDet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random

# Generating a synthetic dataset for emotion analysis in customer feedback
np.random.seed(0)

# Sample data
data_size = 500  # Number of data points
emotions = ['happy', 'angry', 'sad', 'neutral']
feedback_examples = [
    "I love this product, it's absolutely fantastic!",
    "This is the worst experience I've ever had.",
    "I'm so sad about this service, it didn't meet my expectations.",
    "The product is fine, nothing special though."
]

# Creating the dataset
data = {
    "Feedback": [random.choice(feedback_examples) for _ in range(data_size)],
    "Emotion": [random.choice(emotions) for _ in range(data_size)]
}

# Creating a DataFrame
feedback_df = pd.DataFrame(data)

# Displaying the first few rows of the dataset
print(feedback_df.head())


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report


In [ ]:
# Replace with your dataset path
dataset_path = '/content/Customer_Feedback_Emotions.csv'
data = pd.read_csv(dataset_path)


In [ ]:
tfidf = TfidfVectorizer(max_features=2000)
X = tfidf.fit_transform(data['Feedback']).toarray()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Emotion'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes))


In [ ]:
!pip install flask-ngrok


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

@app.route("/predict", methods=["POST"])
def predict():
    text = request.json.get("text", "")
    features = tfidf.transform([text]).toarray()
    prediction = model.predict(features)
    emotion = label_encoder.inverse_transform([np.argmax(prediction)])[0]
    return jsonify({"emotion": emotion})

if __name__ == '__main__':
    app.run()
